**Validando o período de treinamento**

O periodo utilizado para treinar o modelo é um parâmetro importante, o contexto do mercado financeiro no muda a todo momento e os parâmetros do nosso modelo devem ser atualizados.

**Objetivo:**  
- Treinar modelos para 1, 2, 3 e 5 anos de dados históricos e verificar variações no desempenho sem cross validation

**Modelo utilizado:**
- Proporção de treino e teste -> 0.75, 0.25
- Árvore de Decisão com max_depth = 5 e random_state = 42

**Resultados:**
- Treinar o modelo com até 3 anos de dados históricos.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import warnings

import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import aurum

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

warnings.filterwarnings('ignore')

In [40]:
# Funções Helpers:
def get_ohlcv_adapted(TICKER, DIST_ALVO=5, PERIODO=1):
    """_summary_

    Args:
        TICKER (str): Código do ativo
        DIST_ALVO (int, optional): Distancia em dias do alvo. Defaults to 5.
        PERIODO (int, optional): Número de anos usados trazidos. Defaults to 1.

    Returns:
        _type_: _description_
    """

    ticker = yf.Ticker(TICKER) 

    END_DATE = '2022-06-01'
    START_DATE = pd.to_datetime(END_DATE) - pd.DateOffset(365 * PERIODO)

    df = ticker.history(
        start=START_DATE,
        end=END_DATE,
        interval='1d',
    ).reset_index()

    assert DIST_ALVO > 0, 'O número de dias paro o alvo precisa ser maior ou igual a zero.'

    df.loc[:, 'LEAK_Retorno'] = (df['Close'].shift(-DIST_ALVO) - df['Close'])/df['Close']
    df.loc[:, 'Alvo'] = (df['LEAK_Retorno'] > 0.00).astype('int')


    return df

def treinar_modelo(TICKER, PERIODO, DIST_ALVO, TIME_SERIES_SPLIT=False):
    
    import warnings
    warnings.filterwarnings('ignore')

    df = get_ohlcv_adapted(TICKER, PERIODO=PERIODO, DIST_ALVO=DIST_ALVO)
    df = aurum.ft.technical_indicators(df)

    FEATURES = [
        'RSI_14',
        'STOCHd_14_3_3',
        'ROC_2',
        'ROC_5',
        'ROC_10',
        'SLOPE_3',
        'ATRr_5',
        'WILLR_14',
        'OBV_ROC_14',
        'EMA_BUY_CROSS',
        'EMA_SELL_CROSS',
        'EMA_9_DISTANCE',
        'EMA_21_DISTANCE',
        'BBAND_FechouFora_Lower',
        'BBAND_FechouFora_Upper'
    ]

    TARGET = ['Alvo']

    train_size = int(len(df) * .75)

    X_train = df.loc[:train_size, FEATURES]
    y_train = df.loc[:train_size, TARGET]

    X_test = df.loc[train_size:, FEATURES]
    y_test = df.loc[train_size:, TARGET]

    if TIME_SERIES_SPLIT == False:

        est = DecisionTreeClassifier(
            max_depth=5,
            random_state=42,
        )

        est.fit(X_train, y_train)

    else:
        
        PARAM_GRID = {
            'max_depth': [5],
            'random_state': [42]
        }

        model = GridSearchCV(
            DecisionTreeClassifier(),
            param_grid=PARAM_GRID,
            scoring='accuracy',
            cv=TimeSeriesSplit(n_splits=5)
        )

        model.fit(X_train, y_train)

        est=model.best_estimator_


    y_pred = [1 if x == True else 0 for x in (est.predict_proba(X_test)[:, 1] > .5)]

    resultados = df[train_size:].reset_index(drop=True)
    resultados['PREDICOES'] = y_pred
    resultados['RETORNO_MODELO'] = 1 + ((resultados['LEAK_Retorno']/DIST_ALVO) * resultados['PREDICOES'])
    resultados['RETORNO_ACUMULADO_MODELO'] = resultados['RETORNO_MODELO'].cumprod()
    resultados['RETORNO_ACUMULADO_BNH'] = 1 + (resultados['Close'] - resultados.loc[0, 'Close'])/resultados.loc[0, 'Close']

    retorno = pd.DataFrame(columns=['Ticker', 'Periodo', 'DistAlvo', 'RetornoModelo', 'RetornoBnH'])
    retorno.loc[0, 'Ticker'] = TICKER
    retorno.loc[0, 'Periodo'] = PERIODO
    retorno.loc[0, 'DistAlvo'] = DIST_ALVO
    retorno.loc[0, 'RetornoModelo'] = resultados.RETORNO_ACUMULADO_MODELO.tail(1).values[0]
    retorno.loc[0, 'RetornoBnH'] = resultados.RETORNO_ACUMULADO_BNH.tail(1).values[0]

    return retorno

def full_test(LISTA_TICKERS, TIME_SERIES_SPLIT=False):

    resultados = pd.DataFrame()
    for TICKER in LISTA_TICKERS:
        for PERIODO in [1, 2, 3, 4, 5]:
            for DIST_ALVO in [2,4,5,8,10]:
                resultados = pd.concat([resultados, treinar_modelo(TICKER, PERIODO=PERIODO, DIST_ALVO=DIST_ALVO, TIME_SERIES_SPLIT=TIME_SERIES_SPLIT)])

    return resultados

In [22]:
LISTA_ATIVOS = [
    'ABEV3.SA',
    'BBAS3.SA',
    'BEEF3.SA',
    'CMIG4.SA',
    'CSAN3.SA',
    'EQTL3.SA',
    'GGBR4.SA',
    'ITSA4.SA',
    'ITUB4.SA',
    'PETR3.SA',
    'RADL3.SA',
    'SANB11.SA',
    'SUZB3.SA',
    'TAEE11.SA',
    'VALE3.SA',
    'WEGE3.SA'
]

In [37]:
teste_periodo = full_test(LISTA_ATIVOS)

In [53]:
res_test = teste_periodo.groupby(['Periodo', 'DistAlvo']).mean()
res_test['Diff'] = res_test['RetornoModelo'] - res_test['RetornoBnH']
res_test.style.background_gradient(axis=1, subset=['Diff'], cmap='RdBu', vmin=-.3, vmax=.3)

Note que para 3 anos é o periódo que temos o maior retorno positivo em média para o modelo nos ativos selecionados, a maior diferença ocorre a uma distância do alvo de 5 dias.

In [34]:
teste_periodo[['Periodo','DistAlvo','RetornoModelo', 'RetornoBnH']].groupby(['Periodo','DistAlvo']).std()

RetornoModelo  RetornoBnH
Periodo DistAlvo                           
1       2              0.113520    0.164474
        4              0.105463    0.163739
        5              0.105563    0.164658
        8              0.073744    0.158696
        10             0.085313    0.155642
2       2              0.141018    0.162398
        4              0.105680    0.171012
        5              0.097050    0.167374
        8              0.124849    0.155676
        10             0.115402    0.148450
3       2              0.209875    0.187792
        4              0.214797    0.181752
        5              0.282126    0.181326
        8              0.157363    0.184259
        10             0.253691    0.178588
4       2              0.224895    0.289710
        4              0.112601    0.281251
        5              0.126834    0.282842
        8              0.125646    0.297088
        10             0.103220    0.323363
5       2              0.226376    0.244745
        4              0.189431    0.229488
        5              0.191015    0.236189
        8              0.242375    0.251741
        10             0.286925    0.251383

Ao analisarmos o periodo de 3 anos com o alvo em 5 dias, vemos um alto desvio padrão, ou seja, um alto risco para essa estratégia. Isso não necessariamente está diretamente ligado a estratégia em si, o periodo de 3 anos corresponde ao inicio de um Bear Market devido ao Covid-19 e naturalmente a volatilidade/risco de qualquer ativo no periodo era maior.

De modo geral, a **estratégia utilizando o modelo de Machine Learning é pior**, ou seja possui um retorno menor do que o buy and hold, contudo esta **possui um risco menor** dado que o tempo de exposição é menor.

---